<a href="https://colab.research.google.com/github/elizagb/Google-Earth-Engine-Practice/blob/main/FeatureCollection_Prac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import ee

In [3]:
# Trigger the authentication flow.
ee.Authenticate()

In [5]:
# start up the session
ee.Initialize(project='ee-egblack')

In [32]:
# Define the Holiday Farm Fire center
center = ee.Geometry.Point([-122.418960, 44.163945])

# Define a region around the center (e.g., a 20 km buffer)
region = center.buffer(20000)  # 20 km radius

In [33]:
# Load Sentinel-2 surface reflectance imagery for the post-fire period
image = (
    ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
    .filterBounds(region)
    .filterDate("2024-03-01", "2024-09-30")
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 10))  # Filter low-cloud images
    .median()  # Take median composite to reduce noise
    .clip(region)
)

In [34]:
# TODO NEXT: draw polygons for example regions in Google Earth Engines. Use this to get the exact coordinates of my regions. Add these coords in here.

# Define coordinates by class label
landcover_geometries = {
    'river': [
        [[-122.61192320271424,44.13987893440508], [-122.60715959950379,44.13987893440508], [-122.60715959950379,44.14067197353579], [-122.60715959950379, 44.14067197353579], [-122.61192320271424,44.14067197353579]], # Area 1
        # Can add more areas

    ],
    'burn_forest': [
        [[-122.60056964512019,44.160991833255224], [-122.5966643487945,44.160991833255224], [-122.5966643487945,44.16185385813942], [-122.60056964512019,44.16185385813942], [-122.60056964512019,44.160991833255224]],
    ]
}

# Use a loop to build all features
features = []

for label, polygons in landcover_geometries.items():
    for coords in polygons:
        geometry = ee.Geometry.Polygon(coords)
        feature = ee.Feature(geometry, {'landcover': label})
        features.append(feature)

# Create FeatureCollection
training_fc = ee.FeatureCollection(features)


In [35]:
# Confirm that the training areas plot correctly and overlap with the sentinel 2 image region

import geemap
Map = geemap.Map()
Map.centerObject(region, 11)  # Center the map to your region

# Add the image to the map
Map.addLayer(image, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'Image') # Visualize with RGB bands

# Add the training polygons
Map.addLayer(training_fc, {}, 'Training Data')

Map

Map(center=[44.16396551764556, -122.41895920682659], controls=(WidgetControl(options=['position', 'transparent…

In [36]:
# Sample pixels in each polygon and attach the 'landcover' label
training_data = image.sampleRegions(
    collection=training_fc,
    properties=['landcover'],
    scale=10,  # Sentinel-2 resolution
    geometries=True  # Keep geometry for possible visualization
)

# ERROR HERE: size = 0
print(training_data.size().getInfo())
print(training_data.first().getInfo())


0
None


In [16]:
# Train a classifier using the training data

# classifier = RandomForestClassifier